# AIM: 
    1. Get the Total Vechile Count at each intersection for one day
    2. Get the Lane count and total at each intersection for one day 
    3. Find the Lane occupancy 
    4. Map the Geolocations to the final dataframe for visuaization 

In [ ]:
import pandas as pd
import os
#os.chdir("../Task 2")
#Read the combined raw dataset 
Loop_Detector=pd.read_csv(r"D:\Python\AllDay.csv")

Get the Flow observations for each intersection and count the number of lanes. Split and save the vechile count,Lane count for later joining because grouping is done and i need the repeated values

In [2]:
#Getting the sum of RF and MF 
Flow_Observation = Loop_Detector.groupby(['Recorded time','tsc']).agg({'mf1':'sum','mf2':'sum','mf3':'sum','mf4':'sum','mf5':'sum','rf1':'sum','rf2':'sum','rf3':'sum','rf4':'sum','rf5':'sum'})
#Getting the count fo number of lanes
Lane_Count = df.groupby(['Recorded time','tsc']).agg({'mf1':'count','mf2':'count','mf3':'count','mf4':'count','mf5':'count','rf1':'count','rf2':'count','rf3':'count','rf4':'count','rf5':'count'})
Flow_Observation.to_csv("Flow_Observation.csv") 
Lane_Count.to_csv("Lane_Count.csv") 

Calculating the total modified and raw flow obserevd at each intersection for the entire day 

In [5]:
Vechile_Count=pd.read_csv("Flow_Observation.csv")# Read the vechile count 
Total_Modified_Flow = [ row.mf1 + row.mf2 + row.mf3 + row.mf4 + row.mf5 for index, row in Vechile_Count.iterrows() ]
Total_Raw_Flow = [ row.rf1 + row.rf2 + row.rf3 + row.rf4 + row.rf5 for index, row in Vechile_Count.iterrows() ]
Vechile_Count['MF']=Total_Modified_Flow
Vechile_Count['RF']=Total_Raw_Flow
Vechile_Count=Vechile_Count.drop(columns=['mf1', 'mf2','mf3','mf4','mf5','rf1', 'rf2','rf3','rf4','rf5'])

Repeat the process for lane counts

In [6]:
Numer_Of_Lanes=pd.read_csv("Lane_Count.csv")# Read the Lane count 
# Add the needed columns and drop the rest 
mf_LC = [ row.mf1 + row.mf2 + row.mf3 + row.mf4 + row.mf5 for index, row in Numer_Of_Lanes.iterrows() ]
rf_LC = [ row.rf1 + row.rf2 + row.rf3 + row.rf4 + row.rf5 for index, row in Numer_Of_Lanes.iterrows() ]
Numer_Of_Lanes['MF']=mf_LC
Numer_Of_Lanes['RF']=rf_LC
Numer_Of_Lanes=Numer_Of_Lanes.drop(columns=['mf1', 'mf2','mf3','mf4','mf5','rf1', 'rf2','rf3','rf4','rf5','RF','Recorded time'])
Numer_Of_Lanes=Numer_Of_Lanes.rename(columns={"MF": "Lane_Count"}) # rename for inner join between the Lane count datafrae and thevechile count dataframe 

Mapping the Lane count to the dataframe with the vechile count 

In [8]:
Count = dict(zip(Numer_Of_Lanes.tsc, Numer_Of_Lanes.Lane_Count))
Vechile_Count['Lane_Count']=Vechile_Count['tsc']
Vechile_Count['Lane_Count']=Vechile_Count['Lane_Count'].map(Count)

#### Dataframe (Vechile_Count_Lane_Count) 
contains one minute time series of the vechile count and the Lane count

In [10]:
Vechile_Count_Lane_Count=Vechile_Count 

#### Getting the Total Vechile Count for One Day (TVC)

In [12]:
Total_Vechile_Count = VC_LC.groupby('tsc').agg({'MF':'sum','RF':'sum'})
Total_Vechile_Count.to_csv("Total_Vechile_Count_One_Day.csv")
Total_Vechile_Count_With_Lane_Count=pd.read_csv("Total_Vechile_Count_One_Day.csv")
Total_Vechile_Count_With_Lane_Count['Lane_Count']=Total_Vechile_Count_With_Lane_Count['tsc']
Total_Vechile_Count_With_Lane_Count['Lane_Count']=Total_Vechile_Count_With_Lane_Count['Lane_Count'].map(Count)
Visualizing_Data=Total_Vechile_Count_With_Lane_Count #Final Dataframe 

#### Lane Occupancy at each intersection = Total_Vechile_Count / Number_Of_Intersection_One_Day


In [14]:
Visualizing_Data['MF_Over_LaneCount']=" "
Visualizing_Data['RF_Over_LaneCount']=" "
for i in range(len(Visualizing_Data)):
    Visualizing_Data.at[i,'MF_Over_LaneCount']=Visualizing_Data.at[i,'MF']/Visualizing_Data.at[i,'Lane_Count']
    Visualizing_Data.at[i,'RF_Over_LaneCount']=Visualizing_Data.at[i,'RF']/Visualizing_Data.at[i,'Lane_Count']

#### Mapping rules for Location:
    1.Make a dictinory having a key-value pair of the data needed to be mapped
    2.Create a column for which the data must be mapped in the original dataset
    3.Use the Map function and feed the dictinoray as an input

In [16]:
Loaction=pd.read_csv("IntDetailsOutput.csv") # Mapping for Lat Long 
Latitude = dict(zip(Loaction.tsc, Loaction.latitude))
Longitude = dict(zip(Loaction.tsc, Loaction.longitude))
Visualizing_Data['Latitude']=" "
Visualizing_Data['Longitude']=" "
Visualizing_Data['Latitude']=Visualizing_Data['tsc']
Visualizing_Data['Longitude']=Visualizing_Data['tsc']
Visualizing_Data['Latitude']=Visualizing_Data['Latitude'].map(Latitude)
Visualizing_Data['Longitude']=Visualizing_Data['Longitude'].map(Longitude)
Visualizing_Data.to_csv("Visualizing_Data.csv")